In [1]:
pip install networkx matplotlib

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.1 MB 254.3 kB/s eta 0:00:07
   ---------- ----------------------------- 0.5/2.1 MB 254.3 kB/s eta 0:00:07
   ---------- ----------------------------- 0.5/2.1 MB 254.3 kB/s eta 0:00:07
   ----

In [5]:
pip install imageio

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [6]:
import gradio as gr
import networkx as nx
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import imageio
import tempfile
import os

# ---------------- BST ---------------- #

class Node:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None


class BST:
    def __init__(self):
        self.root = None

    def insert(self, val):
        if not self.root:
            self.root = Node(val)
            return

        curr = self.root

        while True:
            if val < curr.val:
                if curr.left:
                    curr = curr.left
                else:
                    curr.left = Node(val)
                    return
            elif val > curr.val:
                if curr.right:
                    curr = curr.right
                else:
                    curr.right = Node(val)
                    return
            else:
                return

    def search_path(self, val):
        curr = self.root
        path = []

        while curr:
            path.append(curr.val)

            if val == curr.val:
                return path, True
            elif val < curr.val:
                curr = curr.left
            else:
                curr = curr.right

        return path, False


# ---------------- AUTO BUILD ---------------- #

def build_balanced_bst():
    bst = BST()

    def helper(arr):
        if not arr:
            return
        mid = len(arr)//2
        bst.insert(arr[mid])
        helper(arr[:mid])
        helper(arr[mid+1:])

    helper(list(range(11)))
    return bst


bst = build_balanced_bst()

# ---------------- GRAPH ---------------- #

def build_graph(root):
    G = nx.DiGraph()

    def add(node):
        if not node:
            return

        if node.left:
            G.add_edge(node.val, node.left.val)
            add(node.left)

        if node.right:
            G.add_edge(node.val, node.right.val)
            add(node.right)

    add(root)
    return G


def hierarchy_pos(G, root, width=1., vert_gap=0.2, vert_loc=0):
    pos = {root: (0, vert_loc)}

    def helper(node, left, right, vert_loc):
        children = list(G.successors(node))
        if not children:
            return
        
        dx = (right-left)/len(children)
        nextx = left + dx/2

        for child in children:
            pos[child] = (nextx, vert_loc-vert_gap)
            helper(child, nextx-dx/2, nextx+dx/2, vert_loc-vert_gap)
            nextx += dx

    helper(root, -width, width, vert_loc)
    return pos


# ---------------- DRAW FRAME ---------------- #

def draw_frame(highlight=None):
    fig, ax = plt.subplots(figsize=(7,5))

    G = build_graph(bst.root)
    pos = hierarchy_pos(G, bst.root.val)

    colors = []
    for node in G.nodes():
        if highlight and node in highlight:
            colors.append("#ff914d")  # orange highlight
        else:
            colors.append("#7fb3ff")  # blue

    nx.draw(
        G,
        pos,
        with_labels=True,
        node_size=2200,
        node_color=colors,
        arrows=False,
        font_weight='bold',
        ax=ax
    )

    fig.canvas.draw()

    img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)

    return img


# ---------------- CREATE GIF ---------------- #

def create_search_animation(val):

    if val is None:
        return None, "Enter a number!"

    path, found = bst.search_path(int(val))

    frames = []

    for step in range(1, len(path)+1):
        frames.append(draw_frame(highlight=path[:step]))

    # pause on final frame
    for _ in range(5):
        frames.append(frames[-1])

    temp_gif = tempfile.NamedTemporaryFile(delete=False, suffix=".gif")
    imageio.mimsave(temp_gif.name, frames, duration=0.8)

    status = "✅ Found!" if found else "❌ Not Found"

    return temp_gif.name, f"Search Path: {' → '.join(map(str,path))}\n{status}"


def show_initial_tree():
    frame = draw_frame()
    temp_gif = tempfile.NamedTemporaryFile(delete=False, suffix=".gif")
    imageio.mimsave(temp_gif.name, [frame], duration=1)

    return temp_gif.name, "✅ Balanced BST created with numbers 0–10."


# ---------------- GRADIO UI ---------------- #

with gr.Blocks(theme=gr.themes.Soft()) as app:

    gr.Markdown("# 🌳 Binary Search Tree — LIVE Animation")
    gr.Markdown("Watch the BST traversal happen automatically!")

    animation = gr.Image(type="filepath", label="BST Animation")

    status = gr.Textbox(label="Status")

    with gr.Row():
        number = gr.Number(label="Enter number to search", precision=0)
        btn = gr.Button("Search 🔍")

    app.load(show_initial_tree, outputs=[animation, status])
    btn.click(create_search_animation, inputs=number, outputs=[animation, status])


app.launch()


C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\4290607408.py:195: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as app:


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\4290607408.py:151: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\4290607408.py:151: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\4290607408.py:151: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)


In [ ]:
import gradio as gr
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import imageio
import tempfile

# ---------------- BST ---------------- #

class Node:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None


class BST:
    def __init__(self):
        self.root = None

    def insert(self, val):
        if not self.root:
            self.root = Node(val)
            return

        curr = self.root

        while True:
            if val < curr.val:
                if curr.left:
                    curr = curr.left
                else:
                    curr.left = Node(val)
                    return
            elif val > curr.val:
                if curr.right:
                    curr = curr.right
                else:
                    curr.right = Node(val)
                    return
            else:
                return


# -------- BUILD PERFECTLY BALANCED TREE -------- #

def build_balanced():
    bst = BST()

    def helper(arr):
        if not arr:
            return
        mid = len(arr)//2
        bst.insert(arr[mid])
        helper(arr[:mid])
        helper(arr[mid+1:])

    helper(list(range(11)))
    return bst


bst = build_balanced()

# current pointer (for guessing)
current_node = bst.root


# ---------------- GRAPH ---------------- #

def build_graph(root):
    G = nx.DiGraph()

    def add(node):
        if not node:
            return

        if node.left:
            G.add_edge(node.val, node.left.val)
            add(node.left)

        if node.right:
            G.add_edge(node.val, node.right.val)
            add(node.right)

    add(root)
    return G


def hierarchy_pos(G, root, width=1., vert_gap=0.2, vert_loc=0):
    pos = {root:(0,vert_loc)}

    def helper(node, left, right, vert_loc):
        children = list(G.successors(node))
        if not children:
            return
        
        dx = (right-left)/len(children)
        nextx = left + dx/2

        for child in children:
            pos[child] = (nextx, vert_loc-vert_gap)
            helper(child, nextx-dx/2, nextx+dx/2, vert_loc-vert_gap)
            nextx += dx

    helper(root, -width, width, vert_loc)
    return pos


# ---------------- DRAW FRAME ---------------- #

def draw_frame(highlight=None):
    fig, ax = plt.subplots(figsize=(7,5))

    G = build_graph(bst.root)
    pos = hierarchy_pos(G, bst.root.val)

    colors = []
    for node in G.nodes():
        if highlight and node == highlight:
            colors.append("#ff914d")
        else:
            colors.append("#7fb3ff")

    nx.draw(
        G,
        pos,
        with_labels=True,
        node_size=2200,
        node_color=colors,
        arrows=False,
        font_weight='bold',
        ax=ax
    )

    fig.canvas.draw()

    img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)

    return img


def make_gif(frame):
    temp = tempfile.NamedTemporaryFile(delete=False, suffix=".gif")
    imageio.mimsave(temp.name, [frame], duration=0.8)
    return temp.name


# ---------------- GAME LOGIC ---------------- #

def start_game():
    global current_node
    current_node = bst.root

    gif = make_gif(draw_frame(current_node.val))

    question = f"🤔 Is your number **{current_node.val}**?"

    return gif, question


def go_left():
    global current_node

    if current_node.left:
        current_node = current_node.left

    gif = make_gif(draw_frame(current_node.val))

    return gif, f"Is your number {current_node.val}?"


def go_right():
    global current_node

    if current_node.right:
        current_node = current_node.right

    gif = make_gif(draw_frame(current_node.val))

    return gif, f"Is your number {current_node.val}?"


def correct():
    gif = make_gif(draw_frame(current_node.val))
    return gif, f"🎉 I guessed it! Your number is {current_node.val}."


# ---------------- UI ---------------- #

with gr.Blocks(theme=gr.themes.Soft()) as app:

    gr.Markdown("# 🌳 BST Guessing AI")
    gr.Markdown("Think of a number from **0–10** and let the tree guess it!")

    animation = gr.Image(type="filepath", height=400)
    question_box = gr.Textbox(label="BST Question")

    with gr.Row():
        start_btn = gr.Button("Start Game 🎯")

    with gr.Row():
        less_btn = gr.Button("⬅️ Lesser")
        equal_btn = gr.Button("✅ Equal")
        greater_btn = gr.Button("Greater ➡️")

    start_btn.click(start_game, outputs=[animation, question_box])
    less_btn.click(go_left, outputs=[animation, question_box])
    greater_btn.click(go_right, outputs=[animation, question_box])
    equal_btn.click(correct, outputs=[animation, question_box])

app.launch()


C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\334005027.py:193: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as app:


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\334005027.py:137: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\334005027.py:137: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\334005027.py:137: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\334005027.py:137: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be rem

In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import imageio
import tempfile

# ---------------- BST ---------------- #

class Node:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None


class BST:
    def __init__(self):
        self.root = None

    def insert(self, val):
        if not self.root:
            self.root = Node(val)
            return

        curr = self.root

        while True:
            if val < curr.val:
                if curr.left:
                    curr = curr.left
                else:
                    curr.left = Node(val)
                    return
            elif val > curr.val:
                if curr.right:
                    curr = curr.right
                else:
                    curr.right = Node(val)
                    return
            else:
                return


# -------- PERFECT BALANCED BST -------- #

def build_balanced():
    bst = BST()

    def helper(arr):
        if not arr:
            return
        mid = len(arr)//2
        bst.insert(arr[mid])
        helper(arr[:mid])
        helper(arr[mid+1:])

    helper(list(range(11)))
    return bst


bst = build_balanced()
current_node = bst.root


# ---------------- TRUE TREE DRAWING ---------------- #

def get_positions(node, depth=0, pos_dict={}, x=0, dx=8):
    """
    Recursively assigns positions so left is left,
    right is right — like a REAL BST.
    """
    if node is None:
        return

    pos_dict[node] = (x, -depth)

    if node.left:
        get_positions(node.left, depth+1, pos_dict, x-dx/(depth+1), dx)

    if node.right:
        get_positions(node.right, depth+1, pos_dict, x+dx/(depth+1), dx)

    return pos_dict


def draw_tree(highlight=None):
    fig, ax = plt.subplots(figsize=(10,6))
    ax.axis('off')

    pos = get_positions(bst.root, dx=8)

    # Draw edges first
    for node, (x, y) in pos.items():

        if node.left:
            x2, y2 = pos[node.left]
            ax.plot([x, x2], [y, y2], linewidth=2)

        if node.right:
            x2, y2 = pos[node.right]
            ax.plot([x, x2], [y, y2], linewidth=2)

    # Draw nodes
    for node, (x, y) in pos.items():

        color = "#ff914d" if highlight == node else "#7fb3ff"

        circle = plt.Circle((x, y), 0.35, color=color)
        ax.add_patch(circle)

        ax.text(
            x, y,
            str(node.val),
            ha='center',
            va='center',
            color='black',
            fontweight='bold'
        )

    fig.canvas.draw()

    img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)

    return img


def make_gif(frame):
    temp = tempfile.NamedTemporaryFile(delete=False, suffix=".gif")
    imageio.mimsave(temp.name, [frame], duration=0.8)
    return temp.name


# ---------------- GAME LOGIC ---------------- #

def start_game():
    global current_node
    current_node = bst.root

    gif = make_gif(draw_tree(current_node))

    return gif, f"🤔 Is your number {current_node.val}?"


def go_left():
    global current_node

    if current_node.left:
        current_node = current_node.left

    gif = make_gif(draw_tree(current_node))

    return gif, f"Is your number {current_node.val}?"


def go_right():
    global current_node

    if current_node.right:
        current_node = current_node.right

    gif = make_gif(draw_tree(current_node))

    return gif, f"Is your number {current_node.val}?"


def correct():
    gif = make_gif(draw_tree(current_node))

    return gif, f"🎉 I guessed it! Your number is {current_node.val}."


# ---------------- UI ---------------- #

with gr.Blocks(theme=gr.themes.Soft()) as app:

    gr.Markdown("# 🌳 TRUE Binary Search Tree Guessing AI")
    gr.Markdown("Think of a number from **0–10** and let the BST find it.")

    animation = gr.Image(type="filepath", height=500)
    question = gr.Textbox(label="BST Question")

    with gr.Row():
        start = gr.Button("Start Game 🎯")

    with gr.Row():
        lesser = gr.Button("⬅️ Lesser")
        equal = gr.Button("✅ Equal")
        greater = gr.Button("Greater ➡️")

    start.click(start_game, outputs=[animation, question])
    lesser.click(go_left, outputs=[animation, question])
    greater.click(go_right, outputs=[animation, question])
    equal.click(correct, outputs=[animation, question])

app.launch()


C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\4196362635.py:177: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as app:


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\4196362635.py:122: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\4196362635.py:122: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\4196362635.py:122: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\4196362635.py:122: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be

In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import imageio
import tempfile

# ---------------- BST ---------------- #

class Node:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None


class BST:
    def __init__(self):
        self.root = None

    def insert(self, val):
        if not self.root:
            self.root = Node(val)
            return

        curr = self.root

        while True:
            if val < curr.val:
                if curr.left:
                    curr = curr.left
                else:
                    curr.left = Node(val)
                    return
            elif val > curr.val:
                if curr.right:
                    curr = curr.right
                else:
                    curr.right = Node(val)
                    return
            else:
                return


# -------- PERFECT BALANCED BST -------- #

def build_balanced():
    bst = BST()

    def helper(arr):
        if not arr:
            return
        mid = len(arr)//2
        bst.insert(arr[mid])
        helper(arr[:mid])
        helper(arr[mid+1:])

    helper(list(range(11)))
    return bst


bst = build_balanced()
current_node = bst.root


# ---------------- PERFECT TREE LAYOUT ---------------- #

def compute_positions(root):
    """
    Computes PERFECT symmetric positions using inorder traversal.
    This guarantees:
    ✔ no overlap
    ✔ equal spacing
    ✔ textbook structure
    """

    pos = {}
    x_counter = [0]

    def inorder(node, depth):
        if not node:
            return

        inorder(node.left, depth + 1)

        pos[node] = (x_counter[0], -depth)
        x_counter[0] += 1

        inorder(node.right, depth + 1)

    inorder(root, 0)
    return pos


# ---------------- DRAW TREE ---------------- #

def draw_tree(highlight=None):

    pos = compute_positions(bst.root)

    fig_width = max(10, len(pos) * 1.2)
    fig, ax = plt.subplots(figsize=(fig_width, 6))

    ax.axis('off')

    # Draw edges FIRST
    for node, (x, y) in pos.items():

        if node.left:
            x2, y2 = pos[node.left]
            ax.plot([x, x2], [y, y2], linewidth=2, color="#555")

        if node.right:
            x2, y2 = pos[node.right]
            ax.plot([x, x2], [y, y2], linewidth=2, color="#555")

    # Draw nodes
    for node, (x, y) in pos.items():

        if node == highlight:
            color = "#ff7f50"  # highlight
        else:
            color = "#4a90e2"  # professional blue

        circle = plt.Circle((x, y), 0.35, color=color)
        ax.add_patch(circle)

        ax.text(
            x,
            y,
            str(node.val),
            ha='center',
            va='center',
            color='white',
            fontsize=12,
            fontweight='bold'
        )

    ax.set_ylim(min(y for _, y in pos.values()) - 1, 1)

    fig.canvas.draw()

    img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)

    return img


def make_gif(frame):
    temp = tempfile.NamedTemporaryFile(delete=False, suffix=".gif")
    imageio.mimsave(temp.name, [frame], duration=0.8)
    return temp.name


# ---------------- GUESSING LOGIC ---------------- #

def start_game():
    global current_node
    current_node = bst.root

    return make_gif(draw_tree(current_node)), f"Is your number {current_node.val}?"


def lesser():
    global current_node

    if current_node.left:
        current_node = current_node.left

    return make_gif(draw_tree(current_node)), f"Is your number {current_node.val}?"


def greater():
    global current_node

    if current_node.right:
        current_node = current_node.right

    return make_gif(draw_tree(current_node)), f"Is your number {current_node.val}?"


def equal():
    return make_gif(draw_tree(current_node)), f"🎉 Got it! Your number is {current_node.val}."


# ---------------- UI ---------------- #

with gr.Blocks(theme=gr.themes.Soft()) as app:

    gr.Markdown("# 🌳 Professional BST Guessing AI")
    gr.Markdown("Think of a number between **0–10** and let the Binary Search Tree guess it.")

    image = gr.Image(type="filepath", height=500)
    question = gr.Textbox(label="BST Decision")

    start_btn = gr.Button("Start")

    with gr.Row():
        lesser_btn = gr.Button("⬅ Lesser")
        equal_btn = gr.Button("✅ Equal")
        greater_btn = gr.Button("Greater ➡")

    start_btn.click(start_game, outputs=[image, question])
    lesser_btn.click(lesser, outputs=[image, question])
    greater_btn.click(greater, outputs=[image, question])
    equal_btn.click(equal, outputs=[image, question])

app.launch()


C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\4268888989.py:189: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as app:


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\4268888989.py:142: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
C:\Users\hisha\AppData\Local\Temp\ipykernel_14380\4268888989.py:142: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
